In [10]:
%pwd

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [12]:
#Load the data.
train = pd.read_json("../input/train.json")

In [13]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
y_train = train['is_iceberg']
median = train[train['inc_angle'] != 'na']['inc_angle'].median()
X_angle = train.inc_angle.replace('na', median)

# Resize image https://stackoverflow.com/questions/41758385/resizing-images-in-keras-imagedatagenerator-flow-methods
# Using scipy.ndimage.zoom instead of scipy.misc.imresize
#import scipy.ndimage
#print(X_train.shape)
#scaling_factor = 1
#new_shape = (X_train.shape[1]*1, X_train.shape[2]*1, X_train.shape[3])
#X_train_new = np.empty(shape=(X_train.shape[0],)+new_shape)
#for idx in range(X_train.shape[0]):
#    X_train_new[idx] = scipy.ndimage.zoom(X_train[idx], zoom=(scaling_factor, scaling_factor, 1), order=1)  #upscale using bilinear interpolation
#print(X_train_new.shape)
#X_train = X_train_new

#target_train=train['is_iceberg']
#X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75, test_size=0.25)

In [14]:
import keras
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import load_model, Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [15]:
def TinyYoloModel(input_shape=(75,75,3)):
    """
    Implementation of the Tiny YOLOv2 Model in Keras.

    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=input_shape)

    #https://github.com/xslittlegrass/CarND-Vehicle-Detection/blob/master/vehicle%20detection.ipynb
    # Layer 1-8
    for i in range(1,9):
        if i==1:
            X = X_input

        # 2D Convolution
        if i != 8:
            X = Conv2D(2**(3+i), kernel_size=(3,3), strides=(1,1), padding='same', name='conv2d_'+str(i), use_bias=False)(X)
        elif i == 8:
            X = Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same', name='conv2d_'+str(i), use_bias=False)(X)

        # Batch Normalization
        X = BatchNormalization(name='batch_normalization_'+str(i))(X)

        # Leaky ReLU
        X = LeakyReLU(alpha=0.1, name='leaky_re_lu_'+str(i))(X)

        # MaxPooling2D
        if i <= 5:
            X = MaxPooling2D(pool_size=(2,2), name='max_pooling_2d_'+str(i))(X)
        if i >= 6:
            X = MaxPooling2D(pool_size=(1,1), name='max_pooling_2d_'+str(i))(X)

    # Layer 9
    X = Conv2D(256, kernel_size=(1,1), strides=(1,1), name='conv2d_9')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    
    
    ## Adding vector incidence angle input
    X_angle_input = Input(shape=(1,))
    
    # Combine image and inc angle
    X2 = keras.layers.concatenate([X, X_angle_input])
    
    X2 = Dense(512)(X2)
    X2 = Activation('relu')(X2)
    X2 = Dense(256)(X2)
    X2 = Activation('relu')(X2)
    X2 = Dense(1, activation='sigmoid')(X2)   
    
    
    #X = Dense(512)(X)
    #X = Activation('relu')(X)
    #X = Dense(256)(X)
    #X = Activation('relu')(X)
    #X = Dense(1, activation='sigmoid')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    #model = Model(inputs = X_input, outputs = X, name='TinyYoloModel')
    model = Model(inputs=[X_input, X_angle_input], outputs=X2, name='TinyYoloModel')

    return model

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=50)

In [16]:
tinyYoloModel = TinyYoloModel(input_shape=X_train.shape[1:])
tinyYoloModel.summary()

num_gpus = 1
if num_gpus > 1:
    from keras.utils import multi_gpu_model
    tinyYoloModel = multi_gpu_model(tinyYoloModel, gpus=num_gpus)
    
tinyYoloModel.compile(optimizer="Adadelta", loss="binary_crossentropy", metrics=['accuracy'])

In [17]:
#%% https://github.com/deepsense-ai/intel-ai-webinar-neural-networks/blob/master/live_loss_plot.py
import matplotlib.pyplot as plt
from keras.callbacks import Callback
from IPython.display import clear_output
#from matplotlib.ticker import FormatStrFormatter

def translate_metric(x):
    translations = {'acc': "Accuracy", 'loss': "Log-loss (cost function)"}
    if x in translations:
        return translations[x]
    else:
        return x

class PlotLosses(Callback):
    def __init__(self, figsize=None):
        super(PlotLosses, self).__init__()
        self.figsize = figsize

    def on_train_begin(self, logs={}):

        self.base_metrics = [metric for metric in self.params['metrics'] if not metric.startswith('val_')]
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)

        clear_output(wait=True)
        plt.figure(figsize=self.figsize)
        
        for metric_id, metric in enumerate(self.base_metrics):
            plt.subplot(1, len(self.base_metrics), metric_id + 1)
            
            plt.plot(range(1, len(self.logs) + 1),
                     [log[metric] for log in self.logs],
                     label="training")
            if self.params['do_validation']:
                plt.plot(range(1, len(self.logs) + 1),
                         [log['val_' + metric] for log in self.logs],
                         label="validation")
            plt.title(translate_metric(metric))
            plt.xlabel('epoch')
            plt.legend(loc='center left')
        
        plt.tight_layout()
        plt.show();

plot_losses = PlotLosses(figsize=(16, 4))

In [18]:
#%% Augmented Image data
seed=123
datagen = ImageDataGenerator(featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.01)
#datagen.fit(X_train_cv, seed=seed)
datagen.fit(X_train, seed=seed)

In [ ]:
batch_size=24*num_gpus
epochs=25

#tinyYoloModel.fit_generator(datagen.flow(X_train_cv, y_train_cv, batch_size=batch_size, seed=seed), validation_data=datagen.flow(X_valid, y_valid, batch_size=batch_size, seed=seed), epochs=10, verbose=1, callbacks=[plot_losses])
#tinyYoloModel.fit_generator(datagen.flow(X_train_cv, y_train_cv, batch_size=batch_size, seed=seed), validation_data=datagen.flow(X_valid, y_valid, batch_size=batch_size, seed=seed), epochs=epochs-10, verbose=1, callbacks=[plot_losses])
#tinyYoloModel.fit(x=X_train_cv, y=y_train_cv, epochs=epochs, verbose=1, batch_size=batch_size, validation_data=(X_valid, y_valid), callbacks=[plot_losses])
tinyYoloModel.fit(x=[X_train, X_angle], y=y_train, epochs=epochs, verbose=1, batch_size=batch_size, validation_split=0.25, callbacks=[plot_losses])

In [ ]:
tinyYoloModel.save_weights("final_model_weights.hdf5")

In [ ]:
test = pd.read_json("../input/test.json")

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=tinyYoloModel.predict(X_test)

In [ ]:
pd.DataFrame(np.around(predicted_test, decimals=2))

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)